In [1]:
import random
import torch
import numpy as np
import gym
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch import nn
from collections import deque # this python module implements exactly what we need for the replay memeory

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
trained = True

In [3]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

Training device: cpu


In [4]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity) # Define a queue with maxlen "capacity"

    def push(self, state, action, next_state, reward):
        self.memory.append((state, action, next_state, reward))

    def sample(self, batch_size):
        batch_size = min(batch_size, len(self)) # Get all the samples if the requested batch_size is higher than the number of sample currently in the memory
        return random.sample(self.memory, batch_size) # Randomly select "batch_size" samples

    def __len__(self):
        return len(self.memory) # Return the number of samples currently stored in the memory

### Policy Network

In [5]:
class DQN(nn.Module):

    def __init__(self, state_space_dim, action_space_dim):
        super().__init__()

        self.linear = nn.Sequential(
            nn.Linear(state_space_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, action_space_dim))

    def forward(self, x):
        return self.linear(x)

In [6]:
# Define an example network
net = DQN(state_space_dim=4, action_space_dim=2)

### $\epsilon$-greedy policy

In [7]:
def choose_action_epsilon_greedy(net, state, epsilon):
    
    if epsilon > 1 or epsilon < 0:
        raise Exception('The epsilon value must be between 0 and 1')
                
    # Evaluate the network output from the current state
    with torch.no_grad():
        net.eval()
        state = torch.tensor(state, dtype=torch.float32) # Convert the state to tensor
        net_out = net(state)

    # Get the best action (argmax of the network output)
    best_action = int(net_out.argmax())
    # Get the number of possible actions
    action_space_dim = net_out.shape[-1]

    # Select a non optimal action with probability epsilon, otherwise choose the best action
    if random.random() < epsilon:
        # List of non-optimal actions
        non_optimal_actions = [a for a in range(action_space_dim) if a != best_action]
        # Select randomly
        action = random.choice(non_optimal_actions)
    else:
        # Select best action
        action = best_action
        
    return action, net_out.numpy()

### Exploration profile

In [ ]:
### Define exploration profile
initial_value = 2.5
max_value = 1
num_iterations = 500
exp_decay = np.exp(-np.log(initial_value) / num_iterations * 10) # We compute the exponential decay in such a way the shape of the exploration profile does not depend on the number of iterations
exploration_profile = [max_value * (exp_decay ** i) for i in range(num_iterations)]

### Plot exploration profile
plt.figure(figsize=(12,8))
plt.plot(exploration_profile)
plt.grid()
plt.xlabel('Iteration', fontsize = 14)
plt.ylabel('Exploration profile (Epsilon)', fontsize = 14)
plt.savefig("Exploration_profile_simple_cartpole.pdf", format='pdf')

## CartPole-v1

In [12]:
### Create environment
env = gym.make('CartPole-v1') # Initialize the Gym environment
env.seed(0) # Set a random seed for the environment (reproducible results)

# Get the shapes of the state space (observation_space) and action space (action_space)
state_space_dim = env.observation_space.shape[0]
action_space_dim = env.action_space.n

print(f"STATE SPACE SIZE: {state_space_dim}")
print(f"ACTION SPACE SIZE: {action_space_dim}")

STATE SPACE SIZE: 4
ACTION SPACE SIZE: 2


### Network update

In [10]:
# Set random seeds
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

### PARAMETERS
gamma = 0.999   # gamma parameter for the long term reward
replay_memory_capacity = 10000   # Replay memory capacity
lr = 5e-4   # Optimizer learning rate
target_net_update_steps = 5   # Number of episodes to wait before updating the target network
batch_size = 64   # Number of samples to take from the replay memory for each update
bad_state_penalty = -100   # Penalty to the reward when we are in a bad state (in this case when the pole falls down) 
min_samples_for_training = 1000   # Minimum samples in the replay memory to enable the training

In [13]:
### Initialize the replay memory
replay_mem = ReplayMemory(replay_memory_capacity)    

### Initialize the policy network
policy_net = DQN(state_space_dim, action_space_dim)

### Initialize the target network with the same weights of the policy network
target_net = DQN(state_space_dim, action_space_dim)
target_net.load_state_dict(policy_net.state_dict()) # This will copy the weights of the policy network to the target network

### Initialize the optimizer
optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr) # The optimizer will update ONLY the parameters of the policy network

### Initialize the loss function
loss_fn = nn.SmoothL1Loss()

### Update function

In [ ]:
def update_step(policy_net, target_net, replay_mem, gamma, optimizer, loss_fn, batch_size):
        
    # Sample the data from the replay memory
    batch = replay_mem.sample(batch_size)
    batch_size = len(batch)
    # Create tensors for each element of the batch
    states      = torch.tensor(np.array([s[0] for s in batch]), dtype=torch.float32)
    actions     = torch.tensor(np.array([s[1] for s in batch]), dtype=torch.int64)
    rewards     = torch.tensor(np.array([s[3] for s in batch]), dtype=torch.float32)

    # Compute a mask of non-final states (all the elements where the next state is not None)
    non_final_next_states = torch.tensor(np.array([s[2] for s in batch if s[2] is not None]), dtype=torch.float32) # the next state can be None if the game has ended
    non_final_mask = torch.tensor([s[2] is not None for s in batch], dtype=torch.bool)

    # Compute all the Q values (forward pass)
    policy_net.train()
    q_values = policy_net(states)
    # Select the proper Q value for the corresponding action taken Q(s_t, a)
    state_action_values = q_values.gather(1, actions.unsqueeze(1))

    # Compute the value function of the next states using the target network V(s_{t+1}) = max_a( Q_target(s_{t+1}, a)) )
    with torch.no_grad():
        target_net.eval()
        q_values_target = target_net(non_final_next_states)
    # For the terminal state the expected reward is zero otherwise is set to the max computed with the target net
    next_state_max_q_values = torch.zeros(batch_size)
    next_state_max_q_values[non_final_mask] = q_values_target.max(dim=1)[0]

    # Compute the expected Q values
    expected_state_action_values = rewards + (next_state_max_q_values * gamma)
    expected_state_action_values = expected_state_action_values.unsqueeze(1) # Set the required tensor shape

    # Compute the loss
    loss = loss_fn(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # Apply gradient clipping (clip all the gradients greater than 2 for training stability)
    nn.utils.clip_grad_norm_(policy_net.parameters(), 2)
    optimizer.step()
    return loss

### Training Loop

In [ ]:
if not trained:  
    # Initialize the Gym environment
    env = gym.make('CartPole-v1') 
    env.seed(0) # Set a random seed for the environment (reproducible results)

    returns, mean_ret = [], []
    patience = 100
    for episode_num, epsilon in enumerate(tqdm(exploration_profile)):

        # Reset the environment and get the initial state
        state = env.reset()
        # Reset the score. The final score will be the total amount of steps before the pole falls
        score = 0
        done = False

        episode_loss = []

        # Go on until the pole falls off
        while not done:

            # Choose the action following the policy
            action, q_values = choose_action_epsilon_greedy(policy_net, state, epsilon)

            # Apply the action and get the next state, the reward and a flag "done" that is True if the game is ended
            next_state, reward, done, info = env.step(action)

            # We apply a (linear) penalty when the cart is far from center
            pos_weight = 1
            reward = reward - pos_weight * np.abs(state[0]) 

            # We apply also a penalty on the pole angle to stabilize better the system
            angle_weight = 0.5
            reward = reward - angle_weight*np.abs(state[3])

            # Update the final score (+1 for each step)
            score += 1

            # Apply penalty for bad state
            if done: # if the pole has fallen down 
                reward += bad_state_penalty
                next_state = None
                returns.append(score)
                mean_ret.append(sum(returns[-100:]) / len(returns[-100:]))
                if mean_ret[-1] >= 475:
                    patience -= 1
                else: patience = 100

            # Update the replay memory
            replay_mem.push(state, action, next_state, reward)
            # Update the network
            if len(replay_mem) > min_samples_for_training: # we enable the training only if we have enough samples in the replay memory, otherwise the training will use the same samples too often
                loss = update_step(policy_net, target_net, replay_mem, gamma, optimizer, loss_fn, batch_size)
                episode_loss.append(loss.detach().numpy())
            # Visually render the environment (disable to speed up the training)
            #env.render()

            # Set the current state for the next iteration
            state = next_state

        if patience == 0: 
            print("The task has been solved after " + str(episode_num) + " episodes")
            break

        # Update the target network every target_net_update_steps episodes
        if episode_num % target_net_update_steps == 0:
            print('Updating target network...')
            target_net.load_state_dict(policy_net.state_dict()) # This will copy the weights of the policy network to the target network
            torch.save(target_net.state_dict(), 'Cartpole-DQL.torch')

        # Print the final score
        print(f"EPISODE: {episode_num + 1} - FINAL SCORE: {score} - Epsilon: {epsilon}") # Print the final score

    # Plot the results
    plt.figure(figsize=(12,8))
    plt.plot(mean_ret, 'r')
    plt.plot(returns, 'b', alpha = 0.3)
    plt.ylabel('Score', fontsize=18)
    plt.xlabel('Episode', fontsize=18)
    plt.savefig("Score_cartpole.pdf", format='pdf')
    env.close()

In [14]:
# Initialize the Gym environment
env = gym.make('CartPole-v1') 
env.seed(0) # Set a random seed for the environment (reproducible results)
returns = []
policy_net.load_state_dict(torch.load('Cartpole-DQL.torch'))
for num_episode in range(100): 
    # Reset the environment and get the initial state
    state = env.reset()
    # Reset the score. The final score will be the total amount of steps before the pole falls
    score = 0
    done = False
    # Go on until the pole falls off or the score reach 490
    while not done:
      # Choose the best action (epsilon 0)
      action, q_values = choose_action_epsilon_greedy(policy_net, state, epsilon=0)
      # Apply the action and get the next state, the reward and a flag "done" that is True if the game is ended
      next_state, reward, done, info = env.step(action)
      # Visually render the environment
      env.render()
      # Update the final score (+1 for each step)
      score += reward 
      # Set the current state for the next iteration
      state = next_state
      # Check if the episode ended (the pole fell down)
    returns.append(score)
    print(f"EPISODE {num_episode + 1} - FINAL SCORE: {score}") 
print(f"MEAN SCORE OVER 100 EPISODES IS: {np.mean(returns)}")
env.close()

2022-03-15 15:46:01.121 python[14794:1220052] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9d014ea390>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-15 15:46:01.122 python[14794:1220052] Warning: Expected min height of view: (<NSButton: 0x7f9d014fc6c0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-15 15:46:01.124 python[14794:1220052] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9d02854580>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-15 15:46:01.126 python[14794:1220052] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9d033fd060>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


EPISODE 1 - FINAL SCORE: 500.0
EPISODE 2 - FINAL SCORE: 500.0
EPISODE 3 - FINAL SCORE: 500.0
EPISODE 4 - FINAL SCORE: 500.0
EPISODE 5 - FINAL SCORE: 500.0
EPISODE 6 - FINAL SCORE: 500.0
EPISODE 7 - FINAL SCORE: 500.0
EPISODE 8 - FINAL SCORE: 500.0
EPISODE 9 - FINAL SCORE: 500.0
EPISODE 10 - FINAL SCORE: 500.0
EPISODE 11 - FINAL SCORE: 500.0
EPISODE 12 - FINAL SCORE: 500.0
EPISODE 13 - FINAL SCORE: 500.0
EPISODE 14 - FINAL SCORE: 500.0
EPISODE 15 - FINAL SCORE: 500.0
EPISODE 16 - FINAL SCORE: 500.0
EPISODE 17 - FINAL SCORE: 500.0
EPISODE 18 - FINAL SCORE: 500.0
EPISODE 19 - FINAL SCORE: 500.0
EPISODE 20 - FINAL SCORE: 500.0
EPISODE 21 - FINAL SCORE: 500.0
EPISODE 22 - FINAL SCORE: 500.0
EPISODE 23 - FINAL SCORE: 500.0
EPISODE 24 - FINAL SCORE: 500.0
EPISODE 25 - FINAL SCORE: 500.0
EPISODE 26 - FINAL SCORE: 500.0
EPISODE 27 - FINAL SCORE: 500.0
EPISODE 28 - FINAL SCORE: 500.0
EPISODE 29 - FINAL SCORE: 500.0
EPISODE 30 - FINAL SCORE: 500.0
EPISODE 31 - FINAL SCORE: 500.0
EPISODE 32 - FINA